In [1]:
# importation des bibliotheques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report


In [2]:
# importation des donnees
df = pd.read_csv('./heart_2020_cleaned.csv')
df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
319791,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
319792,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
319793,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No


In [3]:
# le pretraitement de donnees
df['HeartDisease'] = df["HeartDisease"].map({"Yes":1, "No":0})
df["Smoking"] = df["Smoking"].map({"Yes":1, "No":0})
df["AlcoholDrinking"] = df["AlcoholDrinking"].map({"Yes":1, "No":0})
df['DiffWalking'] = df['DiffWalking'].map({"Yes":1, "No":0})
df['Diabetic'] = df['Diabetic'].map({"Yes":1, "No":0, 'No, borderline diabetes':2, 'Yes (during pregnancy)':3})
df["KidneyDisease"] = df["KidneyDisease"].map({"Yes":1, "No":0})
df["PhysicalActivity"] = df["PhysicalActivity"].map({"Yes":1, "No":0})
df["Asthma"] = df["Asthma"].map({"Yes":1, "No":0})
df['Sex'] = df["Sex"].map({'Female':0, 'Male':1})
df["Stroke"] = df["Stroke"].map({"Yes":1, "No":0})
df['AgeCategory'] = df['AgeCategory'].map({'18-24':13, "25-29":12, "30-34":11, '35-39':10,
                                          "40-44":9, "45-49":8, "50-54":7, "55-59":6, "60-64":5,
                                          "65-69":4, "70-74":3, "75-79":2, '80 or older':1})
df['Race'] = df["Race"].map({'White':0, 'Black':1, 'Asian':3, 'American Indian/Alaskan Native':2,
       'Other':4, 'Hispanic':5})
df["GenHealth"] = df["GenHealth"].map({'Very good':0, 'Fair':1, 'Good':2, 'Poor':3, 'Excellent':4})
df["SkinCancer"] = df["SkinCancer"].map({"Yes":1, "No":0})

In [4]:
# valeurs nulles
df.isna().sum()


HeartDisease        0
BMI                 0
Smoking             0
AlcoholDrinking     0
Stroke              0
PhysicalHealth      0
MentalHealth        0
DiffWalking         0
Sex                 0
AgeCategory         0
Race                0
Diabetic            0
PhysicalActivity    0
GenHealth           0
SleepTime           0
Asthma              0
KidneyDisease       0
SkinCancer          0
dtype: int64

In [5]:
# importation des bibliotheques pour la division des donnees
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from imblearn.over_sampling import SMOTE

In [6]:
seed = 500
# lecture du dataset

df_copy = df.copy()

#Surechantillonnage
sm = SMOTE(k_neighbors=3, sampling_strategy=0.75)

y = df_copy['HeartDisease']
X = df_copy.drop('HeartDisease', axis=1)

x_re, y_re = sm.fit_resample(X, y)

X_train, x_test, y_train, y_test = train_test_split(x_re, y_re, test_size=0.3, random_state=seed)

In [7]:
# la regression logistique
from sklearn.linear_model import LogisticRegression

logis_model = LogisticRegression(random_state=seed)

logis_model.fit(X_train, y_train)

c:\Users\evil smile\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=500)

In [32]:
# evaluation du modele

# prediction sur les futures donnees
y_pred_logi = logis_model.predict(x_test)
# acuracy score
print("\n accuracy score: ", accuracy_score(y_test, y_pred_logi), '\n')
print("-"*70)
# matrice de confusion
print("\n Matrice de confusion:\n\n",confusion_matrix(y_test, y_pred_logi))
print("-"*70)
# rapport de classification
print("\n Rapport de classification:\n\n",classification_report(y_test, y_pred_logi))


 accuracy score:  0.7427795364833705 

----------------------------------------------------------------------

 Matrice de confusion:

 [[67737 19772]
 [19717 46296]]
----------------------------------------------------------------------

 Rapport de classification:

               precision    recall  f1-score   support

           0       0.77      0.77      0.77     87509
           1       0.70      0.70      0.70     66013

    accuracy                           0.74    153522
   macro avg       0.74      0.74      0.74    153522
weighted avg       0.74      0.74      0.74    153522



In [33]:
from xgboost import XGBClassifier
xg_model = XGBClassifier(learning_rate=0.01, n_estimators=25,
                          max_depth=15,gamma=0.6,subsample=0.52,
                          colsample_bytree=0.6,seed=27, reg_lambda=2, 
                          booster='dart', colsample_bylevel=0.6, 
                          colsample_bynode=0.5)

xg_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='dart', callbacks=None,
              colsample_bylevel=0.6, colsample_bynode=0.5, colsample_bytree=0.6,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.6, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=25, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [34]:
# evaluation du modele

# prediction sur les futures donnees
y_pred_xg = xg_model.predict(x_test)
# acuracy score
print("\n accuracy score: ", accuracy_score(y_test, y_pred_xg), '\n')
print("-"*70)
# matrice de confusion
print("\n Matrice de confusion:\n\n",confusion_matrix(y_test, y_pred_xg))
print("-"*70)
# rapport de classification
print("\n Rapport de classification:\n\n",classification_report(y_test, y_pred_xg))


 accuracy score:  0.8016701189406079 

----------------------------------------------------------------------

 Matrice de confusion:

 [[71528 15981]
 [14467 51546]]
----------------------------------------------------------------------

 Rapport de classification:

               precision    recall  f1-score   support

           0       0.83      0.82      0.82     87509
           1       0.76      0.78      0.77     66013

    accuracy                           0.80    153522
   macro avg       0.80      0.80      0.80    153522
weighted avg       0.80      0.80      0.80    153522



In [35]:
from sklearn.naive_bayes import GaussianNB

gnb_model = GaussianNB()

gnb_model.fit(X_train, y_train)

GaussianNB()

In [36]:
# evaluation du modele

# prediction sur les futures donnees
y_pred_gnb = gnb_model.predict(x_test)
# acuracy score
print("\n accuracy score: ", accuracy_score(y_test, y_pred_gnb), '\n')
print("-"*70)
# matrice de confusion
print("\n Matrice de confusion:\n\n",confusion_matrix(y_test, y_pred_gnb))
print("-"*70)
# rapport de classification
print("\n Rapport de classification:\n\n",classification_report(y_test, y_pred_gnb))


 accuracy score:  0.7112465965789919 

----------------------------------------------------------------------

 Matrice de confusion:

 [[65686 21823]
 [22507 43506]]
----------------------------------------------------------------------

 Rapport de classification:

               precision    recall  f1-score   support

           0       0.74      0.75      0.75     87509
           1       0.67      0.66      0.66     66013

    accuracy                           0.71    153522
   macro avg       0.71      0.70      0.71    153522
weighted avg       0.71      0.71      0.71    153522



In [37]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5)

knn_model.fit(X_train, y_train)

KNeighborsClassifier()

In [38]:
# evaluation du modele

# prediction sur les futures donnees
y_pred_knn = knn_model.predict(x_test)
# acuracy score
print("\n accuracy score: ", accuracy_score(y_test, y_pred_knn), '\n')
print("-"*70)
# matrice de confusion
print("\n Matrice de confusion:\n\n",confusion_matrix(y_test, y_pred_knn))
print("-"*70)
# rapport de classification
print("\n Rapport de classification:\n\n",classification_report(y_test, y_pred_knn))


 accuracy score:  0.8485624210210915 

----------------------------------------------------------------------

 Matrice de confusion:

 [[67852 19657]
 [ 3592 62421]]
----------------------------------------------------------------------

 Rapport de classification:

               precision    recall  f1-score   support

           0       0.95      0.78      0.85     87509
           1       0.76      0.95      0.84     66013

    accuracy                           0.85    153522
   macro avg       0.86      0.86      0.85    153522
weighted avg       0.87      0.85      0.85    153522



In [39]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state=seed)

dt_model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=500)

In [40]:
# evaluation du modele

# prediction sur les futures donnees
y_pred_dt = dt_model.predict(x_test)
# acuracy score
print("\n accuracy score: ", accuracy_score(y_test, y_pred_dt), '\n')
print("-"*70)
# matrice de confusion
print("\n Matrice de confusion:\n\n",confusion_matrix(y_test, y_pred_dt))
print("-"*70)
# rapport de classification
print("\n Rapport de classification:\n\n",classification_report(y_test, y_pred_dt))


 accuracy score:  0.8650095751748935 

----------------------------------------------------------------------

 Matrice de confusion:

 [[75980 11529]
 [ 9195 56818]]
----------------------------------------------------------------------

 Rapport de classification:

               precision    recall  f1-score   support

           0       0.89      0.87      0.88     87509
           1       0.83      0.86      0.85     66013

    accuracy                           0.87    153522
   macro avg       0.86      0.86      0.86    153522
weighted avg       0.87      0.87      0.87    153522



In [41]:
from sklearn.ensemble import ExtraTreesClassifier

etc_model = ExtraTreesClassifier(random_state=seed)

etc_model.fit(X_train, y_train)

ExtraTreesClassifier(random_state=500)

In [42]:
# evaluation du modele

# prediction sur les futures donnees
y_pred_etc = etc_model.predict(x_test)
# acuracy score
print("\n accuracy score: ", accuracy_score(y_test, y_pred_etc), '\n')
print("-"*70)
# matrice de confusion
print("\n Matrice de confusion:\n\n",confusion_matrix(y_test, y_pred_etc))
print("-"*70)
# rapport de classification
print("\n Rapport de classification:\n\n",classification_report(y_test, y_pred_etc))


 accuracy score:  0.8900613592840114 

----------------------------------------------------------------------

 Matrice de confusion:

 [[78365  9144]
 [ 7734 58279]]
----------------------------------------------------------------------

 Rapport de classification:

               precision    recall  f1-score   support

           0       0.91      0.90      0.90     87509
           1       0.86      0.88      0.87     66013

    accuracy                           0.89    153522
   macro avg       0.89      0.89      0.89    153522
weighted avg       0.89      0.89      0.89    153522



In [9]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=seed)

rf_model.fit(X_train,y_train)


RandomForestClassifier(random_state=500)

In [10]:
# evaluation du modele

# prediction sur les futures donnees
y_pred_rf = rf_model.predict(x_test)
# acuracy score
print("\n accuracy score: ", accuracy_score(y_test, y_pred_rf), '\n')
print("-"*70)
# matrice de confusion
print("\n Matrice de confusion:\n\n",confusion_matrix(y_test, y_pred_rf))
print("-"*70)
# rapport de classification
print("\n Rapport de classification:\n\n",classification_report(y_test, y_pred_rf))


 accuracy score:  0.9020856945584346 

----------------------------------------------------------------------

 Matrice de confusion:

 [[79414  8095]
 [ 6937 59076]]
----------------------------------------------------------------------

 Rapport de classification:

               precision    recall  f1-score   support

           0       0.92      0.91      0.91     87509
           1       0.88      0.89      0.89     66013

    accuracy                           0.90    153522
   macro avg       0.90      0.90      0.90    153522
weighted avg       0.90      0.90      0.90    153522



In [11]:
from sklearn.model_selection import GridSearchCV
# optimisation du modele de foret aleatoire

#Parameter to be choosen

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [2,4]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the param grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

grid_model = GridSearchCV(rf_model, param_grid=param_grid, cv=5, n_jobs=2)

grid_model.fit(X_train, y_train)

c:\Users\evil smile\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=500), n_jobs=2,
             param_grid={'bootstrap': [True, False], 'max_depth': [2, 4],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72,
                                          80]})

In [12]:
# evaluation du modele

# prediction sur les futures donnees
y_pred_grid = grid_model.predict(x_test)
# acuracy score
print("\n accuracy score: ", accuracy_score(y_test, y_pred_grid), '\n')
print("-"*70)
# matrice de confusion
print("\n Matrice de confusion:\n\n",confusion_matrix(y_test, y_pred_grid))
print("-"*70)
# rapport de classification
print("\n Rapport de classification:\n\n",classification_report(y_test, y_pred_grid))


 accuracy score:  0.7525631505582262 

----------------------------------------------------------------------

 Matrice de confusion:

 [[67877 19632]
 [18355 47658]]
----------------------------------------------------------------------

 Rapport de classification:

               precision    recall  f1-score   support

           0       0.79      0.78      0.78     87509
           1       0.71      0.72      0.72     66013

    accuracy                           0.75    153522
   macro avg       0.75      0.75      0.75    153522
weighted avg       0.75      0.75      0.75    153522



In [13]:
from sklearn.ensemble import AdaBoostClassifier

ada_model = AdaBoostClassifier(random_state=seed)

ada_model.fit(X_train, y_train)

AdaBoostClassifier(random_state=500)

In [14]:
# evaluation du modele

# prediction sur les futures donnees
y_pred_ada = ada_model.predict(x_test)
# acuracy score
print("\n accuracy score: ", accuracy_score(y_test, y_pred_ada), '\n')
print("-"*70)
# matrice de confusion
print("\n Matrice de confusion:\n\n",confusion_matrix(y_test, y_pred_ada))
print("-"*70)
# rapport de classification
print("\n Rapport de classification:\n\n",classification_report(y_test, y_pred_ada))


 accuracy score:  0.7815166555933352 

----------------------------------------------------------------------

 Matrice de confusion:

 [[70154 17355]
 [16187 49826]]
----------------------------------------------------------------------

 Rapport de classification:

               precision    recall  f1-score   support

           0       0.81      0.80      0.81     87509
           1       0.74      0.75      0.75     66013

    accuracy                           0.78    153522
   macro avg       0.78      0.78      0.78    153522
weighted avg       0.78      0.78      0.78    153522



In [19]:
# donc finalement le meilleur modele est celui des forets aleatoires

# tracage de la courbe d'apprentissage
#from sklearn.model_selection import learning_curve

#sizes, training_scores, testing_scores = learning_curve(RandomForestClassifier(), x_test, y_test, cv=10, scoring='accuracy', train_sizes=np.linspace(0.01, 1.0, 50))

In [22]:
"""plt.figure(figsize=(8,8))
plt.plot(sizes, training_scores.mean(axis=1), "--", label="training score")
plt.plot(sizes, testing_scores.mean(axis=1), label="cross-validation score")
# Drawing plot
plt.title("LEARNING CURVE FOR KNN Classifier")
plt.xlabel("Training Set Size"), 
plt.ylabel("Accuracy Score"),
plt.legend(loc="best")
plt.tight_layout()
plt.show()"""

'plt.figure(figsize=(8,8))\nplt.plot(sizes, training_scores.mean(axis=1), "--", label="training score")\nplt.plot(sizes, testing_scores.mean(axis=1), label="cross-validation score")\n# Drawing plot\nplt.title("LEARNING CURVE FOR KNN Classifier")\nplt.xlabel("Training Set Size"), \nplt.ylabel("Accuracy Score"),\nplt.legend(loc="best")\nplt.tight_layout()\nplt.show()'

In [21]:
# enregistrement du modele
import joblib

joblib.dump(rf_model, "RandomForest_model.pkl")

['RandomForest_model.pkl']